In [1]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA RTX 2000 Ada Generation Laptop GPU
Using device: cuda


In [2]:
# Load MNIST dataset class
from torchvision.datasets import MNIST

# Convert image to tensor (numbers)
from torchvision.transforms import ToTensor

# Create batches of data
from torch.utils.data import DataLoader

# Create training data loader
train_loader = DataLoader(
    MNIST(
        root="Dataset",        # folder to store data
        train=True,            # training data
        download=True,         # download if not present
        transform=ToTensor()   # convert image to tensor
    ),
    batch_size=32,             # images per batch
    shuffle=True               # shuffle data
)

# Get one batch of data
images, labels = next(iter(train_loader))

# Print shapes to understand data
print(images.shape)            # batch, channel, height, width
print(labels.shape)            # batch size


Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting Dataset\MNIST\raw\train-images-idx3-ubyte.gz to Dataset\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100.0%


Extracting Dataset\MNIST\raw\train-labels-idx1-ubyte.gz to Dataset\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100.0%


Extracting Dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to Dataset\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting Dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to Dataset\MNIST\raw

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [3]:
import torch
import torch.nn as nn

# Simple neural network
class DigitModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)  # input → hidden
        self.fc2 = nn.Linear(128, 10)     # hidden → output (0–9)

    def forward(self, x):
        x = x.view(x.size(0), -1)         # flatten image
        x = torch.relu(self.fc1(x))       # activation
        x = self.fc2(x)                   # raw scores
        return x

### Loss Function + Optimizer

In [ ]:
import torch.optim as optim  # optimizers

# Create model object
model = DigitModel()

# Loss function: compares prediction vs correct label
criterion = nn.CrossEntropyLoss()

# Optimizer: updates model weights
optimizer = optim.Adam(
    model.parameters(),      # parameters to update
    lr=0.001                 # learning rate
)

In [4]:
import torch.optim as optim  # optimizers

# Create model object
model = DigitModel()

# Loss function: compares prediction vs correct label
criterion = nn.CrossEntropyLoss()

# Optimizer: updates model weights
optimizer = optim.Adam(
    model.parameters(),      # parameters to update
    lr=0.001                 # learning rate
)

In [5]:
model.train()                    # set model to training mode

for images, labels in train_loader:   # loop over batches

    images = images.view(images.size(0), -1)  # flatten images
                                               
    optimizer.zero_grad()         # clear old gradients

    outputs = model(images)       # forward pass

    loss = criterion(outputs, labels)  # compute loss

    loss.backward()               # compute gradients

    optimizer.step()              # update weights

In [7]:
from torchvision.datasets import MNIST      # MNIST dataset
from torchvision.transforms import ToTensor # convert image to tensor
from torch.utils.data import DataLoader     # data loader

# Create test data loader
test_loader = DataLoader(
    MNIST(
        root="Dataset",        # same dataset folder
        train=False,           # test data
        download=True,         # download if needed
        transform=ToTensor()   # convert image to tensor
    ),
    batch_size=32,             # images per batch
    shuffle=False              # do not shuffle test data
)

In [8]:
model.eval()                       # set model to evaluation mode

correct = 0                        # count correct predictions
total = 0                          # count total samples

with torch.no_grad():              # no gradient needed
    for images, labels in test_loader:   # loop over test data

        images = images.view(images.size(0), -1)  # flatten images

        outputs = model(images)    # forward pass

        _, predicted = torch.max(outputs, 1)  # get predicted class

        total += labels.size(0)    # total samples
        correct += (predicted == labels).sum().item()  # correct ones

# print accuracy
print("Test Accuracy:", correct / total)

Test Accuracy: 0.9517


### Predict Manually

In [23]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F

# Path to your image (change this)
image_path = "fyPhv.jpg"

# Image preprocessing (same style as MNIST)
transform = transforms.Compose([
    transforms.Grayscale(),      # convert to grayscale
    transforms.Resize((28, 28)), # resize to MNIST size
    transforms.ToTensor(),       # convert to tensor (0–1)
    transforms.Lambda(lambda x: 1 - x),  # invert
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load and preprocess image
image = Image.open(image_path)
image = transform(image)

# Add batch dimension
image = image.view(1, -1)        # shape: [1, 784]

model.eval()                     # evaluation mode

with torch.no_grad():            # no gradients
    outputs = model(image)       # model prediction
    probabilities = F.softmax(outputs, dim=1)  # convert to probabilities
    confidence, prediction = torch.max(probabilities, 1)

# Print result
print("Predicted Digit:", prediction.item())
print("Confidence:", round(confidence.item() * 100, 2), "%")


Predicted Digit: 3
Confidence: 100.0 %


In [24]:
# Save trained model weights
torch.save(
    model.state_dict(),   # model parameters (weights)
    "classifier.pth"     # file name
)
